In [2]:
import clip_feature_extractor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import loguniform
import numpy as np

%load_ext autoreload
%autoreload 2

In [ ]:
X_train_CIFAR100, y_train_CIFAR100, X_test_CIFAR100, y_test_CIFAR100 = clip_feature_extractor.get_CIFAR100_features()

Files already downloaded and verified
Files already downloaded and verified
Extracting features from CIFAR100 dataset
Loaded previously extracted features from disk.


In [ ]:
# C = 0.316 provided by OpenAI

log_reg_CIFAR100 = LogisticRegression(random_state=0, max_iter=1000, C=0.316, n_jobs=-1)
log_reg_CIFAR100.fit(X_train_CIFAR100, y_train_CIFAR100)
predictions_CIFAR100 = log_reg_CIFAR100.predict(X_test_CIFAR100)
accuracy_CIFAR100 = np.mean((y_test_CIFAR100 == predictions_CIFAR100).astype(float)) * 100.
print(f"Accuracy = {accuracy_CIFAR100:.3f}")


Accuracy = 80.020


In [3]:
X_train_CIFAR10, y_train_CIFAR10, X_test_CIFAR10, y_test_CIFAR10 = clip_feature_extractor.get_CIFAR10_features()

Files already downloaded and verified
Files already downloaded and verified
Loaded features from disk.


In [4]:
# Create a pipeline with data scaling and logistic regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# Define the parameter distributions with valid solver-penalty combinations
param_distributions = [
    {
        'logreg__penalty': ['l1'],
        'logreg__C': loguniform(1e-4, 1e4),
        'logreg__solver': ['liblinear', 'saga'],
    },
    {
        'logreg__penalty': ['l2'],
        'logreg__C': loguniform(1e-4, 1e4),
        'logreg__solver': ['lbfgs', 'liblinear', 'sag', 'saga'],
    },
    {
        'logreg__penalty': ['elasticnet'],
        'logreg__C': loguniform(1e-4, 1e4),
        'logreg__solver': ['saga'],
        'logreg__l1_ratio': np.linspace(0, 1, 10),
    },
    {
        'logreg__penalty': ['none'],
        'logreg__solver': ['lbfgs', 'sag', 'saga'],
    }
]

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=200,
    cv=2,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1,
    verbose=2
)

# Fit the model on the training data
random_search.fit(X_train_CIFAR10, y_train_CIFAR10)

# Output the best parameters and score
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score: ", random_search.best_score_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
test_score = best_model.score(X_test_CIFAR10, y_test_CIFAR10)
print("Test set accuracy: ", test_score)

Fitting 2 folds for each of 200 candidates, totalling 400 fits
[CV] END logreg__C=0.0006306658668123951, logreg__l1_ratio=0.7777777777777777, logreg__penalty=elasticnet, logreg__solver=saga; total time=  15.4s
[CV] END logreg__C=0.0006306658668123951, logreg__l1_ratio=0.7777777777777777, logreg__penalty=elasticnet, logreg__solver=saga; total time=  17.3s
[CV] END logreg__C=0.00014610865886287216, logreg__l1_ratio=0.1111111111111111, logreg__penalty=elasticnet, logreg__solver=saga; total time=  20.1s
[CV] END ...........logreg__penalty=none, logreg__solver=sag; total time=   0.6s
[CV] END ...........logreg__penalty=none, logreg__solver=sag; total time=   0.5s
[CV] END logreg__C=0.00014610865886287216, logreg__l1_ratio=0.1111111111111111, logreg__penalty=elasticnet, logreg__solver=saga; total time=  20.3s
[CV] END logreg__C=0.00010144487859320233, logreg__penalty=l2, logreg__solver=saga; total time=   7.4s
[CV] END logreg__C=0.00010144487859320233, logreg__penalty=l2, logreg__solver=saga

/home/tonyxdsu/.local/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END logreg__C=235.69148616733443, logreg__l1_ratio=0.7777777777777777, logreg__penalty=elasticnet, logreg__solver=saga; total time=26.0min
[CV] END logreg__C=0.00011390176182186649, logreg__penalty=l2, logreg__solver=lbfgs; total time=   1.9s
[CV] END logreg__C=0.00011390176182186649, logreg__penalty=l2, logreg__solver=lbfgs; total time=   1.9s


/home/tonyxdsu/.local/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END logreg__C=235.69148616733443, logreg__l1_ratio=0.7777777777777777, logreg__penalty=elasticnet, logreg__solver=saga; total time=28.1min
[CV] END logreg__C=1.5783280762132288, logreg__penalty=l1, logreg__solver=liblinear; total time= 2.2min
[CV] END logreg__C=0.0002362246997776369, logreg__penalty=l2, logreg__solver=saga; total time=   9.8s


...

[CV] END logreg__C=18.067190675932803, logreg__l1_ratio=0.4444444444444444, logreg__penalty=elasticnet, logreg__solver=saga; total time=24.3min


/home/tonyxdsu/.local/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END logreg__C=18.067190675932803, logreg__l1_ratio=0.4444444444444444, logreg__penalty=elasticnet, logreg__solver=saga; total time=24.1min


/home/tonyxdsu/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
128 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
68 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tonyxdsu/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tonyxdsu/.local/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tonyxdsu/.local/lib/python3.12/site-packages/sklearn/pipeline.py", line 473, in

Best parameters found:  {'logreg__C': np.float64(0.005982221423148767), 'logreg__penalty': 'l2', 'logreg__solver': 'sag'}
Best cross-validation score:  0.9490400000000001
Test set accuracy:  0.9512
